# Lecture 3
## Multi-Linear Regression

Run the following cell with `Shift + Enter` to watch the video.

In [ ]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/f87efac0" frameborder="0" allow="fullscreen"></iframe>')

In the following we load again the life expectancy dataset and show a pairs
plot of all columns except the first one that contains the country names.

In [ ]:
data <- read.csv(file.path("..", "data", "life_expectancy.csv"))
pairs(data[,2:6])

Let us run a multiple linear regression on a training set and look at the
`summary`. In the first line we select all columns except the first one and
remove all rows that contain a missing value in some column with the command
`na.omit`. The dot `.` in the formula of the linear regression means that we
regress Y on all other columns, i.e. all other columns in the data set are taken
as input variables.

In [ ]:
data <- na.omit(data[,2:6])
idx.train <- sample(nrow(data), 3/4*nrow(data))
data.train <- data[idx.train,]
data.test <- data[-idx.train,]
lm.fit <- lm(LifeExpectancy ~ ., data.train)
summary(lm.fit)

To see how some of the numbers shown by the `summary` command are computed, we
define our own `quality.of.fit` function.

In [ ]:
quality.of.fit <- function(fit, data) {
    RSS <- sum((data$LifeExpectancy - predict(fit, data))^2)
    TSS <- sum((data$LifeExpectancy - mean(data$LifeExpectancy))^2)
    cat(c("Residual standard error: ",
          sqrt(1/(nrow(data) - length(coefficients(fit))) * RSS),
          " R^2 = ", 1 - RSS/TSS, "\n"))
}
quality.of.fit(lm.fit, data.train)
quality.of.fit(lm.fit, data.test)

Compare the outcome obtained here with the values obtained with the `summary`
function in the previous cell.

WARNING: The following 3 cells may run very slowly on the EPFL servers.
There shouldn't be any issue, if you run it locally on decent hardware,
but be prepared to wait when you run these cells on the EPFL servers (or don't
run them at all).

The following code loads the histopathologic cancer detection dataset, and plots
randomly some of the images (shown in the slides).

In [ ]:
load(url("https://lcnwww.epfl.ch/bio322/PCaml.rda"))

par(mfrow = c(4, 6), oma = c(0, 0, 0, 0),
    mar = c(.1, .1, .1, .1), pty="s")
for (i in sample(1:40000, 24)) {
    image(t(1 - matrix(PCaml_x[i,], 32, 32)), col = gray.colors(256),
          axes = FALSE)
}

We prepare the data for fitting with logistic regression.

In [ ]:
data.hpc <- data.frame(PCaml_x)
data.hpc$Y <- PCaml_y
mean(data.hpc$Y)

Next we define a training and a test set, run logistic regression on the
training set (note again that we use the dot `.` in the formula to regress on
all other columns in the data frame, i.e. on all pixel values) and we plot the
ROC curve.

In [ ]:
data.hpc.train <- data.hpc[1:30000,]
data.hpc.test <- data.hpc[30001:40000,]
fit <- glm(Y ~ ., data.hpc.train, family = "binomial")
p <- predict(fit, data.hpc.test, type = "response")
library(ROCR)
plot(performance(prediction(p, data.hpc.test$Y), "tpr", "fpr"))

The quiz questions for this week are summarized in a single quiz.
After this section you should be ready to answer the first two questions of the
[quiz](https://moodle.epfl.ch/mod/quiz/view.php?id=1096745).

## Confidence and Prediction Interval Estimation

In [ ]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/ad995701" frameborder="0" allow="fullscreen"></iframe>')

Let us define a data generator function, draw two training sets and run linear
regression.

In [ ]:
data.generator <- function(b, N, sigma = 0.3) {
    x <- runif(N)
    data.frame(X = x, Y = b[1] + b[2] * x + sigma * rnorm(N))
}
set.seed(1)
b <- c(2, 1.4)
N <- 50
data1 <- data.generator(b, N)
data2 <- data.generator(b, N)
plot(data1, col = "red")
points(data2, col = "blue")
abline(b, lwd = 2)
fit1 <- lm(Y ~ ., data1)
fit2 <- lm(Y ~ ., data2)
abline(fit1, col = "red", lwd = 2)
abline(fit2, col = "blue", lwd = 2)

If we include in the `predict` function the argument `interval = "confidence"`
it will return an array with 3 columns (you may want to look at it with
`head(conf.interval)`). The first column contains the prediction, the second the
lower bound and the third the upper bound of the 95% confidence interval.

In [ ]:
grid <- seq(0, 1, length.out = 100)
conf.interval <- predict(fit1, data.frame(X = grid), interval = "confidence")
plot(data1, col = "red")
abline(b, lwd = 2)
lines(grid, conf.interval[,1], col = "red", lwd = 2)
lines(grid, conf.interval[,2], col = "darkgreen", lwd = 2)
lines(grid, conf.interval[,3], col = "darkgreen", lwd = 2)
abline(fit2, col = "blue", lwd = 2)

If instead we use the argument `interval = "prediction"` we get the prediction
interval.

In [ ]:
pred.interval <- predict(fit1, data.frame(X = grid), interval = "prediction")
plot(data1, col = "red")
abline(b, lwd = 2)
lines(grid, pred.interval[,1], col = "red", lwd = 2)
lines(grid, pred.interval[,2], col = "darkgreen", lwd = 2)
lines(grid, pred.interval[,3], col = "darkgreen", lwd = 2)
points(data2, col = "blue")

Everything we did for the one dimensional case, generalizes of course to
multiple predictors. In the following cell we define a dataset with 3 predictors
and, for the moment, a dummy response `Y` which is initialized with zeros.
Then we use the function `model.matrix` to get the model matrix $\boldsymbol{X}$
shown in the slides.

In [ ]:
set.seed(671)
N <- 10^4
data <- data.frame(X1 = runif(N), X2 = runif(N), X3 = runif(N), Y = rep(0, N))
m <- model.matrix(Y ~ ., data)
head(m)

Next we generate a random coefficient vector and use it to generate the actual
response values $Y$. The operator `%*%` is used to perform matrix
multiplication (i.e. matrix times vector).

In [ ]:
beta <- rnorm(4)
data$Y <- m %*% beta + .2 * rnorm(N)
head(data$Y)

Now we run multiple linear regression.

In [ ]:
idxs <- sample(10^4, 10^3)
data.train <- data[idxs,]
lm.fit <- lm(Y ~ ., data.train)
summary(lm.fit)

You can check that the standard errors shown in the summary above are computed
with the formula shown in the slides. The `solve` function performs the matrix
inversion and the `t` function computes the matrix transform. The estimated
standard deviation of the irreducible error is obtained via the `summary`
function, which computes the `sigma` in the same way as our `quality.of.fit`
function.

In [ ]:
m.train <- model.matrix(Y ~ ., data.train)
sqrt(diag(solve(t(m.train) %*% m.train)))*summary(lm.fit)$sigma

If we want to see the 95% confidence interval of the parameters we can use the
function `confint`.

In [ ]:
confint(lm.fit)

In the following cell we check whether approximately 95% of the test data is
indeed within the confidence interval.

In [ ]:
data.test <- data[-idxs,]
p <- predict(lm.fit, data.test, interval = "prediction")
mean(data.test$Y < p[,3] & data.test$Y > p[,2])

In those last two cells of this section we see that the width of the prediction
interval is indeed computed with the formula given in the slides. We compute the
prediction interval for the 20 first points in the test set.

In [ ]:
p <- predict(lm.fit, data.test[1:20,], interval = "prediction")
p[,3] - p[,1]

Note that the model matrix `m.test` contains as rows the test points in the form
we wanted it, e.g. we could see the first row of this matrix as the $x_0^T$ from
the slides. The factor `qt((1 - .95)/2, lm.fit$df.residual)` used to scale the
standard deviation is approximately equal to 2.

In [ ]:
m.test <- model.matrix(Y ~ ., data.test[1:20,])
qt((1-.95)/2, lm.fit$df.residual)*sqrt(diag(m.test %*% solve(t(m.train) %*% m.train) %*%
t(m.test)) + 1)*summary(lm.fit)$sigma

## Confounders, Heteroscedacity and the Bootstrap

In [ ]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/4f57392d" frameborder="0" allow="fullscreen"></iframe>')

### Confounders
In this example we construct data with correlation between predictors `X1` and
`X2`.

In [ ]:
set.seed(19)
N <- 100
z <- runif(N)
data <- data.frame(X1 = 0.7*z, X2 = z + .01*rnorm(N),
                   X3 = runif(N), Y = rep(0, N))
beta <- c(-.8, -.4, 2, -.5)
m <- model.matrix(Y ~ ., data)
data$Y <- m %*% beta + .1 * rnorm(N)
pairs(data)

You can see that the standard errors for $\beta_1$ and $\beta_2$ are rather
large, because an increase in $\beta_1$ can be compensated with a decrease in
$\beta_2$ and vice versa.

In [ ]:
lm.fit <- lm(Y ~ ., data)
summary(lm.fit)

If we exclude `X2` from the fit, which can be done by writing `- X2` in the
formula, the standard error for $\beta_1$ is narrow. The reason is that we no
longer have the possibility to compensate for a change in $\beta_1$ with
$\beta_2$.

In [ ]:
lm.fit <- lm(Y ~ . - X2, data)
summary(lm.fit)

The follwing code was used to generate the artificial shark attack dataset shown
in the slides.

In [ ]:
icecream <- function(x) {.5*x}
shark_attack <- function(x) {.03*x - 10}
N <- 200
swimmers <- floor(runif(N, 5, 500))
icecream_consumed <- pmax(0, floor(icecream(swimmers) + 20*rnorm(N)))
shark_attacks <- pmax(0, floor(shark_attack(swimmers) + 5*rnorm(N)))
beach_data <- data.frame(swimmers, icecream_consumed, shark_attacks)
pairs(beach_data)

In [ ]:
fit1 = lm(shark_attacks ~ icecream_consumed)
summary(fit1)

In [ ]:
fit2 = lm(shark_attacks ~ swimmers + icecream_consumed)
summary(fit2)

In the following example both the predictor `X1` and the response `Y` are caused
by an unobserved variable `Z`, but `Y` does not directly depend on `X1`.
Of course, in the fit we see that `X1` is related to `Y`.

In [ ]:
set.seed(12)
N <- 100
z <- runif(N)
data <- data.frame(X1 = 0.5*z, X2 = runif(N), X3 = runif(N))
data$Y <- z + .1 * rnorm(N)
summary(lm(Y ~ ., data))

### Heteroscedacity
Let us now generate data with noise that depends on the predictor, i.e.
heteroscedastic noise.

In [ ]:
data.generator1 <- function(b, N) {
    x <- runif(N)
    data.frame(X = x, Y = b[1] + b[2] * x + .1 * x^2 * rnorm(N))
}
set.seed(7)
b <- c(.1, .5)
N <- 50
data <- data.generator1(b, N)
plot(data)

In [ ]:
lm.fit <- lm(Y ~ X, data)
summary(lm.fit)

We see that the prediction interval is too large for small `X` values and too
small for large `X` values.

In [ ]:
grid <- seq(0, 1, length = 100)
pred.interval <- predict(lm.fit, data.frame(X = grid), interval = "prediction")
plot(data)
abline(b)
lines(grid, pred.interval[, 1], col = 'red', lwd = 2)
lines(grid, pred.interval[, 2], col = 'darkgreen')
lines(grid, pred.interval[, 3], col = 'darkgreen')

We prepare now to estimate the standard errors of the coefficients by sampling
many training sets. Our function `fitted.coefficients` returns the coefficients
of a linear regression on some `data`. The `summarize` function takes many
estimated coefficients as input and returns the mean and standard error per
coefficient. We use again the `replicate` function to obtain `10^3` estimated
coefficients with different training sets.

In [ ]:
fitted.coefficients <- function(data) coef(lm(Y ~ ., data))
summarize <- function(coeffs) {
    m <- rowMeans(coeffs)
    B <- ncol(coeffs)
    data.frame(Estimate = m, Std.Error = sqrt(1/(B-1)*rowSums((coeffs - m)^2)))
}
B <- 10^3
summarize(replicate(B, fitted.coefficients(data.generator1(b, N))))

The `bootstrap` function takes a dataset as input and returns a new dataset of
the same size, with rows sampled with replacement from the input dataset. You
can see which rows are duplicated with the row numbers e.g. `43.2` indicates
that this is the third time row `43` is sampled.

In [ ]:
bootstrap <- function(data) data[sample(nrow(data), nrow(data), replace = T),]
bootstrap(data)

Now, without using multiple training sets, we estimate the standard errors of
the coefficients with only a single training set and the bootstrap.

In [ ]:
summarize(replicate(B, fitted.coefficients(bootstrap(data))))

In a similar way we can use the bootstrap to estimate confidence intervals.

In [ ]:
conf.bootstrap <- summarize(replicate(B, predict(lm(Y ~ ., bootstrap(data)), data.frame(X = grid))))
plot(data)
abline(b)
lines(grid, conf.bootstrap[,1], col = "red", lwd = 2, lty = 2)
lines(grid, conf.bootstrap[,1] + 2*conf.bootstrap[,2], col = "darkgreen", lty = 2)
lines(grid, conf.bootstrap[,1] - 2*conf.bootstrap[,2], col = "darkgreen", lty = 2)
lines(grid, conf.interval[, 2], col = 'darkgreen')
lines(grid, conf.interval[, 3], col = 'darkgreen')

In the following cells there are more examples where the bootstrap leads to more
accurate estimates of the standard errors of the coefficients than the linear
regression formulas. In both cases the reason is that the true data generation
process is non-linear and therefore the assumptions used to compute the standard
errors with the formulas do not hold. The bootstrap does not rely on these
assumptions and therefore leads to better estimates, usually.

In [ ]:
data.generator2 <- function(b, N) {
    x <- runif(N)
    data.frame(X = x, Y = b[1] + b[2] * x + b[3] * x^2 + .1 * rnorm(N))
}
set.seed(1)
b <- c(0, -1, 5)
data <- data.generator2(b, N)
plot(data)

In [ ]:
lm.fit <- lm(Y ~ X, data)
summary(lm.fit)

In [ ]:
plot(data)
grid <- seq(0, 1, length.out = 100)
p <- predict(lm.fit, data.frame(X = grid), interval = "prediction")
plot(data)
lines(grid, p[, 1], lwd = 2)
lines(grid, p[, 2], col = 'blue')
lines(grid, p[, 3], col = 'blue')

In [ ]:
summarize(replicate(B, fitted.coefficients(data.generator2(b, N))))

In [ ]:
summarize(replicate(B, fitted.coefficients(bootstrap(data))))

In [ ]:
data.generator3 <- function(b, N) {
    data <- data.frame(X1 = rnorm(N), X2 = rnorm(N), X3 = rnorm(N), Y = rep(0, N))
    m <- model.matrix(Y ~ . + X1*X2, data)
    data$Y <- m %*% b + .01*rnorm(N)
    data
}
N <- 100
set.seed(11)
b <- rnorm(5)
data <- data.generator3(b, N)
pairs(data)
lm.fit <- lm(Y ~ ., data)
summary(lm.fit)$coeff

In [ ]:
summarize(replicate(B, fitted.coefficients(data.generator3(b, N))))

In [ ]:
summarize(replicate(B, fitted.coefficients(bootstrap(data))))

You should now be ready to answer questions 3 to 7 of the
[quiz](https://moodle.epfl.ch/mod/quiz/view.php?id=1096745).

## Loss Functions

In [ ]:
IRdisplay::display_html('<iframe width="640" height="360" src="https://tube.switch.ch/embed/e0a1c22a" frameborder="0" allow="fullscreen"></iframe>')

In the following cells we define the loss functions as the negative
log-likelihood function for logistic and linear regression.

In [ ]:
logistic <- function(x) 1/(1 + exp(-x))
set.seed(12)
b <- c(.5, -1)
N <- 50
x <- runif(N)
data <- data.frame(X = x, Y = logistic(b[1] + b[2]*x) > runif(N))
logistic.loss <- function(b) {
    x <- data$X
    y <- data$Y
    p <- logistic(b[1] + b[2] * x)
    -sum(y * log(p)) - sum((1 - y) * log(1 - p))
}

In [ ]:
plot.loss <- function(loss) {
    grid1 <- seq(-3, 3, length = 20)
    grid2 <- seq(-8, 8, length = 20)
    z <- matrix(apply(expand.grid(grid1, grid2), 1, loss), 20, 20)
    par(mfrow = c(1, 2))
    persp(grid1, grid2, z, phi = 0, theta = 20, xlab = "beta_0", ylab = "beta_1", zlab = "loss")
    contour(grid1, grid2, z, xlab = "beta_0", ylab = "beta_1")
}
plot.loss(logistic.loss)

The negative log-likelihood function for linear regression under the Gaussian
noise assumption would contain some constants that we omit here. Without these
constants the negative log-likelihood function coincides with the sum of squared
errors.

In [ ]:
data <- data.frame(X = x, Y = b[1] + b[2] * x + .2 * rnorm(N))
regression.loss <- function(b) {
    x <- data$X
    y <- data$Y
    p <- b[1] + b[2] * x
    sum((y - p)^2)
}
plot.loss(regression.loss)

Please head over to the final questions of the
[quiz](https://moodle.epfl.ch/mod/quiz/view.php?id=1096745).

## Conceptual Exercises

No programming is required.

**Q1.** I collect a set of data ($n = 100$ observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. $Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \varepsilon$ .

(a) Suppose that the true relationship between $X$ and $Y$ is linear, i.e. $Y = \beta_0 + \beta_1 X + \varepsilon$. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

(b) Answer (a) using test rather than training RSS.

(c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

(d) Answer (c) using test rather than training RSS.

**Q2.** How high is the probability that a given observation is in a given
bootstrap sample? Maybe you want to make a guess, before you solve this
exercise.

We will now derive the probability that a given observation is part of a bootstrap sample. Suppose that we obtain a bootstrap sample from a set of $n$ observations.

(a) What is the probability that the first bootstrap observation is not the jth observation from the original sample? Justify your answer.

(b) What is the probability that the second bootstrap observation is not the jth observation from the original sample ?

(c) Argue that the probability that the jth observation is not in the bootstrap sample is $(1 - 1/n)^n$.

(d) When $n = 5$, what is the probability that the jth observation is in the bootstrap sample ?

(e) When $n = 100$, what is the probability that the jth observation is in the bootstrap sample ?

(f)  When $n = 10000$, what is the probability that the jth observation is in
the bootstrap sample?

(g) What is the probability in the limit $n \to \infty$?

**Q3.** Let us assume the following simple data set with two-dimensional
predictors and categorical responses with 3 categories: $x_1 = (1, 0)$, $y_1 = 1$,
$x_2 = (1, 1)$, $y_2 = 2$, $x_3 = (0, 1)$, $y_3 = 3$.

(a) Compute the loss of multinomial logistic regression if all coefficient
vectors are equal to zero.

(b) Compute the loss of multinomial logistic regression if $\beta_i = x_i$ for
$i = 1, 2, 3$ (we assume implicitly negligible intercepts $\beta_{i0} = 0$).

(c) Compute the posterior probabilities $\mathrm{Pr}(Y = k| X = (0.5, 0))$ for
the coefficient vectors in (a).

## Applied Exercises

**Q4.** In this exercises we will look at confounders, non-linearity and
heteroscedasticity in the life expectancy dataset.

(a) Perform 4 times linear regression by fitting the life expectancy onto year,
GDP, BMI and alcohol consumption separately.

(b) Look at the summary of one of your fits and explain in your own words what
the values in columns `Estimate` and `Std. Error` of the `Coefficients` table mean.

(c) Are the slopes ($\beta_1$) of the separate linear fits smaller, the same or larger
than the corresponding estimated coefficients in the multiple linear regression?
If they differ, explain what could be the reasons for the difference.

(d) Perform a bootstrap estimate of the standard error of the coefficients.

(e) Use the function `predict` to compute the prediction interval for the
response given input Year = 2020, GDP = 80000, BMI = 25, Alcohol = 5. Explain in
your own words what the resulting values mean.
